<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#네이버-뉴스-기사-Scraping-(3.14일-9강-강의-내용)" data-toc-modified-id="네이버-뉴스-기사-Scraping-(3.14일-9강-강의-내용)-1">네이버 뉴스 기사 Scraping (3.14일 9강 강의 내용)</a></span></li><li><span><a href="#NLP-개요" data-toc-modified-id="NLP-개요-2">NLP 개요</a></span><ul class="toc-item"><li><span><a href="#자연어-분석-:-기반-기술" data-toc-modified-id="자연어-분석-:-기반-기술-2.1">자연어 분석 : 기반 기술</a></span></li><li><span><a href="#응용-기술-:-모델을-접목한-도메인-Specific-기술" data-toc-modified-id="응용-기술-:-모델을-접목한-도메인-Specific-기술-2.2">응용 기술 : 모델을 접목한 도메인 Specific 기술</a></span></li></ul></li><li><span><a href="#KoNLPy,-NLTK" data-toc-modified-id="KoNLPy,-NLTK-3">KoNLPy, NLTK</a></span><ul class="toc-item"><li><span><a href="#KoNLPy-패키지-설치" data-toc-modified-id="KoNLPy-패키지-설치-3.1">KoNLPy 패키지 설치</a></span></li><li><span><a href="#KoNLPy-import,-POS-tagging" data-toc-modified-id="KoNLPy-import,-POS-tagging-3.2">KoNLPy import, POS tagging</a></span></li></ul></li></ul></div>

## 네이버 뉴스 기사 Scraping (3.14일 9강 강의 내용)

In [1]:
from functions.download import getDownload    # requests.get() 메서드 구현

In [2]:
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import re
import os, sys
from datetime import datetime
import numpy as np

In [3]:
url = "https://news.naver.com/"

In [4]:
html = getDownload(url)

In [5]:
dom = BeautifulSoup(html.text, "html.parser")

In [6]:
# 카테고리 이름
categoryName = []

for cat in dom.select(".category"):
    for a in cat.find_all("a"):
        #print(a.text.strip())
        categoryName.append(a.text.strip())

categoryName = [name.replace('/', '') for name in categoryName]
categoryName

['정치', '경제', '사회', '생활문화', '세계', 'IT과학']

In [7]:
# 영역별 뉴스 리스트 찾기
news_list = []

for sec in dom.select("[id^='ranking_10']"):    # id가 ranking_100 ~ ranking_105인 태그 아래에 섹션별로 내용이 있음
    sec_news = []
    for li in sec.find_all("li"):
        # print(li.text)
        # print(urljoin(url, li.find("a")["href"]))
        
        # li 태그 아래 em 태그에 순번이 저장되어 있음(순번 제외하고 저장하기 위해, li.find("a").text 사용. 순번까지 저장하려면, li.text 사용)
        sec_news.append([li.find("a").text.strip(), urljoin(url, li.find("a")["href"])])
    
    news_list.append(sec_news)   # 섹션별로 구분해서 제목과 링크를 저장함(3차원 배열 : 섹션, 기사, 제목/링크)

# news
np.shape(news_list)  # 6개 카테고리, 10개 기사, 제목과 링크

(6, 10, 2)

In [8]:
# naver_news 아래에 현재 시간("년-월-일_시간_분")으로 폴더 만들기
now = datetime.now().isoformat()
day_hour_minute = now[:10] + "_" + now[11:13] + "-" + now[14:16]
path = "naver_news/" + day_hour_minute

try:
    if not os.path.exists(path):
        os.makedirs(path)
except:
    raise os.OSError("(%s) 디렉토리를 만들 수 없습니다!" % (path))

# 기사 다운로드 후 본문 저장하기
for section in news_list:
    sectionName = categoryName[news_list.index(section)]

    for sec in section:
        # print(sec[1])   # 0번째에 기사 제목, 1번째에 기사 링크가 있음
        # print(news_list[0][0][0], news_list[0][0][1])
        aid = re.findall(r"aid=\d+", sec[1])[0].split("=")[1]
        fileName = sectionName + "-" + aid + ".txt"        
        
        html = getDownload(sec[1])
        dom = BeautifulSoup(html.text, "html.parser")
        
        content = dom.select("#articleBodyContents")[0]
        # print(dom.select("#articleBodyContents"))
        # print(content.text)
        # print("-" * 50)
        
        fullName = os.path.join(path, fileName)

        # content에 <script> 태그 내의 내용이 포함되어 있음 (삭제)
        content_ = content.text.replace("// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}", "")
 
        with open(fullName, "w") as f:
            f.write(content_)
    
    print(sectionName, "기사 다운로드 완료")

정치 기사 다운로드 완료
경제 기사 다운로드 완료
사회 기사 다운로드 완료
생활문화 기사 다운로드 완료
세계 기사 다운로드 완료
IT과학 기사 다운로드 완료


In [9]:
# 폴더 이름을 포함한 파일명 찾기
full_filenames = []

def search(dirname):
    filenames = os.listdir(dirname)
    
    for filename in filenames:
        full_filename = os.path.join(dirname, filename)
        full_filenames.append(full_filename)
        
    return full_filenames
    
full_filenames = search(path)
#full_filenames

In [10]:
# 저장된 내용 출력
# for full_filename in full_filenames:
#     print(full_filename)
#     print("-" * 50)
    
#     with open(full_filename, "r") as f:
#         print(f.read())
        
#     print(">" * 50, end="\n\n\n")

In [11]:
# 뉴스 기사 리스트 파일로 저장
import pandas as pd

csvFileName = os.path.join(path, "news_list.csv")

news_list_2d = np.array(news_list).reshape(-1, 2)

df = pd.DataFrame(news_list_2d)
df.to_csv(csvFileName, header=False, index=False)

In [13]:
df.head()

,0,1
0,"최선희 北외무성 부상 ""미국과 비핵화 협상 중단 고려""(종합)",https://news.naver.com/main/ranking/read.nhn?m...
1,"'딸 KT 특혜채용' 김성태 ""정치공작"", 여야 ""사실이면 사퇴""(상보)",https://news.naver.com/main/ranking/read.nhn?m...
2,"文대통령, 캄보디아 국왕 환담…한반도 평화 지지 당부도(종합)",https://news.naver.com/main/ranking/read.nhn?m...
3,'선거제 개편' 험로…패스트트랙 마지노선 내 처리는 '불발',https://news.naver.com/main/ranking/read.nhn?m...
4,"'김경수 판사 사퇴' 청원에 靑 ""관여할 수 없고, 해서도 안돼""",https://news.naver.com/main/ranking/read.nhn?m...


## NLP 개요

### 자연어 분석 : 기반 기술

- **형태소 분석** : 토큰 분리, 어간 추출, 품사 부착, 색인, 벡터화
    - 입력된 문장을 형태소 단위로 분류하고 품사를 부착
- **구문 분석** : 문장 경계 인식, 구문분석, 공기어, 개체명 사전 구축(PLOT, 수치, 외국어 한글 표기), 개체명 인식
    - 주어, 목적어, 서술어와 같은 구문단위를 찾음
- **의미 분석** : 대용어 해소(대명사, 두문자어, 약어, 수치), 의미 중의성 해결(동명이인 ,이명이인)
    - 문장이 의미적으로 올바른 문장인지를 판단
- **담화 분석**(담론 분석) : 분류, 군집, 중복, 요약, 가중치, 순위화, 토픽 모델링, 이슈 트래킹, 평판분석, 감성분석, 복합논증분석
    - 대화 흐름상 어떤 의미를 가지는지를 찾음: 문맥 구조 분석(문장들의 연관관계), 의도분석( )
- 중의성 해소

형태소 분석, 구문 분석 분야는 어느 정도 기술이 성숙되어 있으나, 의미 분석, 담화 분석 등은 아직도 연구가 진행중인 분야임

- 담화 : 문장이 연속되어 이루어지는 말의 단위, 문단
- 문장 : 완결된 내용을 나타내는 최소 단위
- 어절 : 문장을 구성하는 단위 (주어, 서술어, 관형어, ...) --> 띄어쓰기 단위
- 단어 : 어절을 구성하는 요소 --> 형태소의 조합으로 구성되어 있음 (관형사, 명사, 조사, 동사) --> 품사를 붙일 수 있는 단위
- 형태소 : (형태론적으로, 문법적으로) 의미를 가진 문법의 최소 단위(어근 분리) --> 형식 형태소(분석에 불필요)와 의존 형태소(분석에 사용)로 구분

**형태소와 단어** : 단어는 뜻을 지니고 홀로 쓰일 수 있는 말, 형태소는 문법상 뜻을 가진 가장 작은 말
    - 자립성에 따라 : 자립 형태소, 의존 형태소
    - 의미에 따라 : 실질 형태소, 형식 형태소
    - 단어 형성 방법에 따른 분류
        - 단일어 : 어근
        - 복합어 : 합성어(어근 + 어근), 파생어( 접두사 + 어근, 어근 + 접미사)

### 응용 기술 : 모델을 접목한 도메인 Specific 기술

- 검색
- 온라인 광고
- 자동번역
- 감정분석
- 음성인식
- 맞춤법검사

## KoNLPy, NLTK

- NLTK는 한국어를 지원하지 않음

### KoNLPy 패키지 설치

- KoNLPy : 한국어 정보처리를 위한 파이썬 패키지 --> Corpus, Morpheme Alalyzer, POS Tagging, ... 
    - https://konlpy-ko.readthedocs.io/ko/v0.4.3/
    - JDK 1.8 이상, JPype1 0.5.7 이상

- konlpy 설치

~~~
$ pip install konlpy     # Python 2.x
$ pip3 install konlpy    # Python 3.x
~~~
    
- MAC에 conda 환경에서 설치 시, 아래와 같은 conda  환경에서는 install 에러가 났으나,

```
$ conda activate nlp      # python3.6.1
$ pip install konlpy
```

- 아래의 conda 환경에서는 정상적으로 install 되었음

```
$ conda activate base    # python3.7.1
$ pip install konlpy
```

- python 3.6.7에서 install error 메시지

```  
Failed building wheel for JPype1
Running setup.py clean for JPype1
Failed to build JPype1
Installing collected packages: JPype1
...
...
Command "/Users/alex/anaconda3/envs/nlp/bin/python -u -c "import setuptools, tokenize;__file__='/private/var/folders/k4/8cb3q8ws6mb4d1tjs8y85tj80000gn/T/pip-install-m1n7e9_x/JPype1/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record /private/var/folders/k4/8cb3q8ws6mb4d1tjs8y85tj80000gn/T/pip-record-thqtqp1b/install-record.txt --single-version-externally-managed --compile" failed with error code 1 in /private/var/folders/k4/8cb3q8ws6mb4d1tjs8y85tj80000gn/T/pip-install-m1n7e9_x/JPype1/
```

### KoNLPy import, POS tagging

In [1]:
from konlpy.tag import Kkma

In [2]:
ma = Kkma()
ma.pos("학교종이 땡땡ㅌ땡")

[('학교', 'NNG'),
 ('종', 'NNG'),
 ('이', 'JKS'),
 ('땡땡', 'MAG'),
 ('ㅌ', 'UN'),
 ('땡', 'MAG')]